In [72]:
import asyncio
from binance import AsyncClient, BinanceSocketManager
import pandas as pd
import datetime as dt
from binance.client import Client

In [73]:
from dotenv import load_dotenv
import os

In [74]:
%run ./Binance_keys.ipynb

In [75]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///CryptoDB.db')

In [76]:
client = Client(api_key,api_secret)

In [77]:
pd.read_sql('SELECT name FROM sqlite_master WHERE type ="table"',engine)

,name
0,ROSEBUSD
1,PONDBUSD
2,BNBBUSD
3,MBOXBUSD
4,ALPHABUSD
...,...
252,BNTBUSD
253,JSTBUSD
254,BTGBUSD
255,USDPBUSD


In [78]:
symbols = pd.read_sql('SELECT name FROM sqlite_master WHERE type ="table"',engine).name.to_list()

In [79]:
#dt.datetime.now() #need set to binance time

In [80]:
#pd.read_sql('BTCBUSD',engine)

In [81]:
now = dt.datetime.now()

In [82]:
now

datetime.datetime(2021, 12, 6, 18, 45, 2, 271132)

In [83]:
def qry(symbol, lookback:int):
    now = dt.datetime.now() - dt.timedelta(hours=0) and dt.timedelta(hours=1) #binance time
    before = now - dt.timedelta(minutes=lookback)
    qry_str = f"""SELECT * FROM '{symbol}' WHERE TIME >= '{before}'"""
    return pd.read_sql(qry_str,engine)

In [84]:
qry('BTCBUSD',5)

,symbol,Time,Price
0,BTCBUSD,2021-12-03 09:42:21.551000,56615.81
1,BTCBUSD,2021-12-03 09:42:23.670000,56615.82
2,BTCBUSD,2021-12-03 09:42:24.242000,56615.82
3,BTCBUSD,2021-12-03 09:42:24.242000,56615.82
4,BTCBUSD,2021-12-03 09:42:24.242000,56615.82
...,...,...,...
168680,BTCBUSD,2021-12-06 07:45:01.923000,48079.38
168681,BTCBUSD,2021-12-06 07:45:01.923000,48079.39
168682,BTCBUSD,2021-12-06 07:45:01.923000,48079.41
168683,BTCBUSD,2021-12-06 07:45:01.934000,48079.41


In [85]:
#loop
rets = []
for symbol in symbols:
    prices = qry(symbol,3).Price
    cumret = (prices.pct_change() +1).prod() - 1
    rets.append(cumret)

In [86]:
top_coin = symbols[rets.index(max(rets))]

In [87]:
top_coin

'VIDTBUSD'

In [88]:
investment_amt = 9

In [89]:
info = client.get_symbol_info(symbol=top_coin)

In [90]:
Lotsize = float([i for i in info['filters'] if i['filterType'] == 'LOT_SIZE'][0]['minQty'])

In [91]:
price = float(client.get_symbol_ticker(symbol=top_coin)['price'])

In [98]:
price

1.1541

In [99]:
Lotsize

1.0

In [100]:
buy_quantity = round(investment_amt/price, len(str(Lotsize).split('.')[1]))

In [101]:
free_usd = [i for i in client.get_account()['balances'] if i['asset'] == 'BUSD'][0]['free']

In [102]:
free_usd

'9.35839200'

In [103]:
if float(free_usd) > investment_amt:
    order = client.create_order(symbol=top_coin, side='BUY',
                                type='MARKET',quantity=buy_quantity)
    print(order)
    
else:
    print('order has not been executed. You are already invested')
    quit()

BinanceAPIException: APIError(code=-1013): Filter failure: LOT_SIZE

In [104]:
buyprice = float(order['fills'][0]['price'])

NameError: name 'order' is not defined

In [ ]:
def createframe(msg):
    df = pd.DataFrame([msg])
    df = df.loc[:,['s','E','p']]
    df.columns = ['symbol','Time','Price']
    df.Price = df.Price.astype(float)
    df.Time = pd.to_datetime(df.Time, unit='ms')
    return df

In [ ]:
async def main(coin):
    bm = BinanceSocketManager(client)
    ts = bm.trade_socket(coin)
    async with ts as tscm:
        while True:
            res = await tscm.recv()
            if res:
                frame = createframe(res)
                if frame.Price[0] <buyprice * 0.97 or frame.Price[0] > 1.005 * buyprice:
                        order = client.create_order(symbol=coin,
                        side='SELL',
                        type='MARKET',
                        quantity=buy_quantity)
                        print(order)
                        loop.stop()
    await client.close_connection()
                            

In [ ]:
if __name__=="__main__":
    
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main(top_coin))